# Imports

In [1]:
import sys
sys.path.append('../')

In [2]:
from evaluation.embedrank_transformers import EmbedRankTransformers

### Evaluate

The following code begins by definning some parameters, such as:
- whether or not to apply "stemming" or "lemmatization" during the normalization pre-processing step; 
- the number of keyphrases to extract;
- the number of folds for the KEA supervised algorithm;
- the list of the datasets from which to extract keyphrases; 
- the path of the dataset (the folder where the datasets can found - `pathData` is defined as `data` on the assumption that the `data` folder is one level above this notebook);
- the path of the output, that is, the path where the trec_eval files (.qrel and .out) should be saved;
- the path where the datasets can be found;
- the path where the Keyphrases are saved;
- if we want to apply stemming on the evaluation side (both on the ground-truth side and on the baselines side). The application of stemming in evaluation mode will increase the effectiveness of the results and is a technique followed by many researchers during the evaluation stage;
- the list of the algorithms;
- the statistical test to apply when comparing the effectiveness of the different algorithms;
- the list of evaluation measures from which to obtain the effectiveness;
- the output format in which we want the results to be presented.

In [7]:
# Some algorithms have a normalization parameter which may be defined with None, stemming or lemmatization
normalization = None  # Other options: "stemming" (porter) and "lemmatization"

# Num of Keyphrases do Retrieve
numOfKeyphrases = 20

# Num of folds for training KEA
nFolds = 5

# List of Datasets
# ListOfDatasets = ['110-PT-BN-KP', '500N-KPCrowd-v1.1', 'citeulike180',
#                   'fao30', 'fao780', 'Inspec', 'kdd', 'Krapivin2009',
#                   'Nguyen2007', 'pak2018', 'PubMed', 'Schutz2008', 'SemEval2010',
#                   'SemEval2017', 'theses100', 'wiki20', 'www', 'cacic', 'wicc', 'WikiNews']
ListOfDatasets = ['Inspec']

# ListOfAlgorithms = ['RAKE', 'YAKE', 'MultiPartiteRank', 'TopicalPageRank', 'TopicRank', 'SingleRank', 'TextRank', 'KPMiner', 'TFIDF', 'KEA']
ListOfAlgorithms = ['YAKE', 'EmbedRankTransformers']

pathData = '../data'
pathOutput = pathData + "/conversor/output/"
pathDataset = pathData + "/Datasets/"
pathKeyphrases = pathData + "/Keywords/"

EvaluationStemming = True #performs stemming when comparing the results obtained from the algorithm with the ground-truth

statistical_test = ["student"] #wilcoxon

measures = ['map','P.5']
   
formatOutput = 'df' #options: 'csv', 'html', 'json', 'latex', 'sql', 'string', 'df'

#### From Jupyter

Next, we initialize the corresponding object (e.g., `YAKE`, etc) and call the `ExtractKeyphrases` and the `Convert2Trec_Eval` methods (the evaluation process will be conducted later on).

For the unsupervised methods, `ExtractKeyphrases()`, will compute the document frenquecy (for the `KPMiner` and `TFIDF`) and the LDA Model (for the `TopicalPageRank`) on top of the dataset defined (in this example `500N-KPCrowd-v1.1`). Recall that the dfs will be saved under the `data/Models/Unsupervised/dfs/NameOfTheDataset_dfs.gz` folder, and the LDA under the `data/Models/Unsupervised/lda/NameOfTheDataset_lda.gz` one. In addition, it will load the entire set of documents from which to extract keyphrases, through the `LoadDatasetFiles`, and apply the `runMultipleDocs` method, which defines, not only the keyphrases folder (i.e., the folder where the keyphrases will be saved), but also runs (internally) the `runSingleDoc` for each individual document.

For the supervised methods, `ExtractKeyphrases()`, will do the following:
- Split the dataset into folds (by calling internally the `CrossValidation` method; 
- Train the model, that is, create a KEA model (model.pickle) and a DF model (df.tsv.gz) for each of the nFolds (based on the training documents). It does this (internally) based on the `TrainingModel` method; 
- Get keyphrases for each of the 5 testing folds, based on the KEA MODEL obtained from the training documents of the corresponding fold. This involves loading (for each fold) the set of documents existing in the testing folder (by applying - internally - the `LoadDatasetFiles` method) and running the `runMultipleDocs` method, which defines, not only the keywords folder (i.e., the folder where the keyphrases will be saved), but also runs (internally) the `runSingleDoc` for each individual document;

Finally, `Convert2Trec_Eval` method will create (internally) the `.qrel` file (ground-truth) and the `.out` file (results by the algorithm).

In [6]:
from keep import Rake
from keep import YAKE
from keep import MultiPartiteRank
from keep import TopicalPageRank
from keep import TopicRank
from keep import PositionRank
from keep import SingleRank
from keep import TextRank
from keep import KPMiner
from keep import TFIDF
from keep import KEA

for algorithm in ListOfAlgorithms:
    print("\n")
    print("----------------------------------------------------------------------------------------")
    print(f"Preparing Evaluation for \033[1m{algorithm}\033[0m algorithm")

    for i in range(len(ListOfDatasets)):
        dataset_name = ListOfDatasets[i]
        print("\n----------------------------------")
        print(f" dataset_name = {dataset_name}")
        print("----------------------------------")

        if algorithm == 'RAKE':
            Rake_object = Rake(numOfKeyphrases, pathData, dataset_name)
            Rake_object.ExtractKeyphrases()
            Rake_object.Convert2Trec_Eval(EvaluationStemming)
        elif algorithm == 'YAKE':
            YAKE_object = YAKE(numOfKeyphrases, pathData, dataset_name)
            YAKE_object.ExtractKeyphrases()
            YAKE_object.Convert2Trec_Eval(EvaluationStemming)
        elif algorithm == 'MultiPartiteRank':
            MultiPartiteRank_object = MultiPartiteRank(numOfKeyphrases, pathData, dataset_name)
            MultiPartiteRank_object.ExtractKeyphrases()
            MultiPartiteRank_object.Convert2Trec_Eval(EvaluationStemming)
        elif algorithm == 'TopicalPageRank':
            TopicalPageRank_object = TopicalPageRank(numOfKeyphrases, pathData, dataset_name, normalization)
            TopicalPageRank_object.ExtractKeyphrases()
            TopicalPageRank_object.Convert2Trec_Eval(EvaluationStemming)
        elif algorithm == 'TopicRank':
            TopicRank_object = TopicRank(numOfKeyphrases, pathData, dataset_name)
            TopicRank_object.ExtractKeyphrases()
            TopicRank_object.Convert2Trec_Eval(EvaluationStemming)
        elif algorithm == 'PositionRank':
            PositionRank_object = PositionRank(numOfKeyphrases, pathData, dataset_name, normalization)
            PositionRank_object.ExtractKeyphrases()
            PositionRank_object.Convert2Trec_Eval(EvaluationStemming)
        elif algorithm == 'SingleRank':
            SingleRank_object = SingleRank(numOfKeyphrases, pathData, dataset_name, normalization)
            SingleRank_object.ExtractKeyphrases()
            SingleRank_object.Convert2Trec_Eval(EvaluationStemming)
        elif algorithm == 'TextRank':
            TextRank_object = TextRank(numOfKeyphrases, pathData, dataset_name, normalization)
            TextRank_object.ExtractKeyphrases()
            TextRank_object.Convert2Trec_Eval(EvaluationStemming)
        elif algorithm == 'KPMiner':
            KPMiner_object = KPMiner(numOfKeyphrases, pathData, dataset_name, normalization)
            KPMiner_object.ExtractKeyphrases()
            KPMiner_object.Convert2Trec_Eval(EvaluationStemming)
        elif algorithm == 'TFIDF':
            TFIDF_object = TFIDF(numOfKeyphrases, pathData, dataset_name, normalization)
            TFIDF_object.ExtractKeyphrases()
            TFIDF_object.Convert2Trec_Eval(EvaluationStemming)
        elif algorithm == 'KEA':
            KEA_object = KEA(numOfKeyphrases, pathData, dataset_name, normalization)
            KEA_object.ExtractKeyphrases(nFolds)
            KEA_object.Convert2Trec_Eval(EvaluationStemming)
        elif algorithm == 'EmbedRankTransformers':
            KEA_object = EmbedRankTransformers(numOfKeyphrases, pathData, dataset_name, normalization)
            KEA_object.ExtractKeyphrases()
            KEA_object.Convert2Trec_Eval(EvaluationStemming)



----------------------------------------------------------------------------------------
Preparing Evaluation for EmbedRankTransformers algorithm

----------------------------------
 dataset_name = Inspec
----------------------------------


-----------------Extract Keyphrases--------------------------

datasetID = Inspec; Number of Files = 2000; Language of the Dataset = en
Keyphrases will be saved in this folder: ../data/Keywords/EmbedRankTransformers/Inspec

Output Keyphrases Folder doesn't exit: ../data/Keywords/EmbedRankTransformers/Inspec. Let's Create it
File: 1/2000

/Users/anz2/anaconda3/envs/ke/lib/python3.7/site-packages/bert_serving/client/__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '
../model/embedrank_transformers.py:248: RuntimeWarning: invalid value encountered in true_divide
  np.max(cosine_similarities) - np.min(cosine_similarities))
../model/embedrank_transformers.py:263: RuntimeWarning: invalid value encountered in true_divide
  np.nanmax(cosine_similarities, axis=0) - np.nanmin(cosine_similarities, axis=0))
../model/embedrank_transformers.py:267: RuntimeWarning: Mean of empty slice
  0.5 + (cosine_sims_norm - np.nanmean(cosine_sims_norm, axis=0)) / np.nanstd(cosine_sims_norm, axis=0)
/Users/anz2/anaconda3/envs/ke/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1667

File: 5/2000

../model/embedrank_transformers.py:248: RuntimeWarning: invalid value encountered in true_divide
  np.max(cosine_similarities) - np.min(cosine_similarities))
../model/embedrank_transformers.py:263: RuntimeWarning: invalid value encountered in true_divide
  np.nanmax(cosine_similarities, axis=0) - np.nanmin(cosine_similarities, axis=0))
../model/embedrank_transformers.py:267: RuntimeWarning: Mean of empty slice
  0.5 + (cosine_sims_norm - np.nanmean(cosine_sims_norm, axis=0)) / np.nanstd(cosine_sims_norm, axis=0)
/Users/anz2/anaconda3/envs/ke/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1667: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/Users/anz2/anaconda3/envs/ke/lib/python3.7/site-packages/bert_serving/client/__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficien

File: 11/2000

...

File: 1993/2000

/Users/anz2/anaconda3/envs/ke/lib/python3.7/site-packages/bert_serving/client/__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '
/Users/anz2/anaconda3/envs/ke/lib/python3.7/site-packages/bert_serving/client/__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


File: 1996/2000

../model/embedrank_transformers.py:248: RuntimeWarning: invalid value encountered in true_divide
  np.max(cosine_similarities) - np.min(cosine_similarities))
../model/embedrank_transformers.py:263: RuntimeWarning: invalid value encountered in true_divide
  np.nanmax(cosine_similarities, axis=0) - np.nanmin(cosine_similarities, axis=0))
../model/embedrank_transformers.py:267: RuntimeWarning: Mean of empty slice
  0.5 + (cosine_sims_norm - np.nanmean(cosine_sims_norm, axis=0)) / np.nanstd(cosine_sims_norm, axis=0)
/Users/anz2/anaconda3/envs/ke/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1667: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/Users/anz2/anaconda3/envs/ke/lib/python3.7/site-packages/bert_serving/client/__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficien

File: 2000/2000
100% of the Extraction Concluded

Convert to trec_eval format
1 - CreateOutFile: ../data/conversor/output/Inspec_EmbedRankTransformers.out
2 - CreateQrelFile: ../data/conversor/output/Inspec.qrel


After extracting the keyphrases and converting the results to a trec_eval format, we are now ready to evaluate the results of the different algorithms.

In [9]:
import os
from IPython.display import display
from keep import SIGTREC_Eval  

for dataset in ListOfDatasets:
    print("\n")
    print("----------------------------------------------------------------------------------------")
    print(f"Running Evaluation for \033[1m{dataset}\033[0m dataset")
    
    path2qrel_file = f"{pathOutput}{dataset}.qrel"
    datasetid = os.path.basename(path2qrel_file)
    
    resultsFiles = []
    for alg in ListOfAlgorithms:
        resultsFiles.append(f"{pathOutput}{dataset}_{alg}.out")

    sig = SIGTREC_Eval()
    results = sig.Evaluate(path2qrel_file, datasetid, resultsFiles, measures, statistical_test, formatOutput)
    
    for res in results:
        if formatOutput == "df":  
            display(res)
        else:
            print(res)



----------------------------------------------------------------------------------------
Running Evaluation for Inspec dataset


#### From Command Prompt

Execution via the command prompt can be done directly from jupyter using the `% run` command (which is the same as typing python in the prompt). In the execution box `$` refers to its variable.

Assuming that you have already loaded the defined parameters from the beginning of the section, you should now run the following code to extract keyphrases from multiple documents.

In [19]:
for algorithm in ListOfAlgorithms:
    for dataset in ListOfDatasets:
        pathD = pathDataset + f"{dataset}"
        pathK = pathKeyphrases + f"{algorithm}/{dataset}"

        #Extract keyphrases
        %run -m keep.cli -pathData $pathData -datasetsName $dataset -algorithm $algorithm -nKeyphrases $numOfKeyphrases -normalization $normalization
        
        #Convert2Trec_Eval Format
        %run -m keep.conversor.cli -pathDatase $pathD -pathKeyphrases $pathK -pathOutput $pathOutput -dataset_name $dataset -algorithmName $algorithm -EvaluationStemming $EvaluationStemming


datasetID = 500N-KPCrowd-v1.1; Number of Files = 500; Language of the Dataset = en
Keyphrases will be saved in this folder: data/Keywords/YAKE/500N-KPCrowd-v1.1

Output Keyphrases Folder already exists.
File: 500/500
100% of the Extraction Concluded
1 - CreateOutFile: data/conversor/output/500N-KPCrowd-v1.1_YAKE.out
2 - CreateQrelFile: data/conversor/output/500N-KPCrowd-v1.1.qrel



datasetID = 500N-KPCrowd-v1.1; Number of Files = 500; Language of the Dataset = en
Keyphrases will be saved in this folder: data/Keywords/Rake/500N-KPCrowd-v1.1

Output Keyphrases Folder already exists.
File: 500/500
100% of the Extraction Concluded
1 - CreateOutFile: data/conversor/output/500N-KPCrowd-v1.1_RAKE.out
2 - CreateQrelFile: data/conversor/output/500N-KPCrowd-v1.1.qrel




After extracting the keyphrases and converting the results to a trec_eval format, we are now ready to evaluate the results of the different algorithms.

In [11]:
for dataset in ListOfDatasets:       
    path2qrel_file = f"{pathOutput}{dataset}.qrel"
    YAKE_OUT = f"{pathOutput}{dataset}_YAKE.out"
    RAKE_OUT = f"{pathOutput}{dataset}_RAKE.out"
    
    %run -m keep.sigtrec_eval.cli -m map P.5 -s student -i $path2qrel_file $YAKE_OUT $RAKE_OUT